## Telangana Growth Analysis

## Datasets From : https://codebasics.io/challenge/codebasics-resume-project-challenge

## By: Uday Kiran Reddy S

In [ ]:
import pandas as pd

In [374]:
fact_stamps = pd.read_csv('/Users/udayreddy/Documents/telangana project files/C7 Input Files_/dataset/fact_stamps.csv')
fact_transport = pd.read_csv('/Users/udayreddy/Documents/telangana project files/C7 Input Files_/dataset/fact_transport.csv')
fact_ts_ipass = pd.read_csv('/Users/udayreddy/Documents/telangana project files/C7 Input Files_/dataset/fact_TS_iPASS.csv')
districts =  pd.read_csv('/Users/udayreddy/Documents/telangana project files/C7 Input Files_/dataset/dim_districts.csv')
dates = pd.read_csv('/Users/udayreddy/Documents/telangana project files/C7 Input Files_/dataset/dim_date.csv')


In [15]:
'''fact_stamps.info()
fact_transport.info()
fact_ts_ipass.info()
districts.info()
dates.info()'''
districts.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 33 entries, 0 to 32
Data columns (total 2 columns):
 #   Column     Non-Null Count  Dtype 
---  ------     --------------  ----- 
 0   dist_code  33 non-null     object
 1   district   33 non-null     object
dtypes: object(2)
memory usage: 660.0+ bytes


In [23]:
#fact_stamps = pd.merge(left=fact_stamps,right=districts,on='dist_code')
fact_stamps.iloc[10:15]

fact_stamps

,dist_code,month,documents_registered_cnt,documents_registered_rev,estamps_challans_cnt,estamps_challans_rev,district
0,14_1,2019-04-01,4533,59236363,0,0,Mahabubnagar
1,14_1,2019-05-01,4301,73344601,0,0,Mahabubnagar
2,14_1,2019-06-01,3565,50331243,0,0,Mahabubnagar
3,14_1,2019-07-01,5056,58994710,0,0,Mahabubnagar
4,14_1,2019-08-01,4864,73289753,0,0,Mahabubnagar
...,...,...,...,...,...,...,...
1499,15_3,2022-11-01,1235,43951677,1284,77098216,Vikarabad
1500,15_3,2022-12-01,1321,72749213,1353,37351018,Vikarabad
1501,15_3,2023-01-01,1118,31875190,1218,34139087,Vikarabad
1502,15_3,2023-02-01,1378,41844930,1427,43546323,Vikarabad


## How does the revenue generated from document registration vary across districts in Telangana? List down the top 5 districts that showedthe highest revenue growth between FY 2019 and 2022.

In [60]:
#fact_stamps = pd.merge(left=fact_stamps,right=dates,on='month')
growth_by_year = fact_stamps.groupby(['district','fiscal_year']).agg({'documents_registered_rev':'sum'}).reset_index()
growth_by_year = growth_by_year.pivot(index='district', columns='fiscal_year',values='documents_registered_rev').reset_index()
growth_by_year['growth_rate_pa']= (growth_by_year[2022] / growth_by_year[2019]) ** (1 / (2022-2019)) - 1
growth_by_year.sort_values('growth_rate_pa',ascending = False).reset_index(drop=True).head(5)

fiscal_year,district,2019,2020,2021,2022,growth_rate_pa
0,Mulugu,80349554,70219240,147262077,217697250,0.394087
1,Adilabad,178082233,167962874,324085539,396840845,0.306167
2,Karimnagar,804402135,600968635,1068655123,1550449161,0.244500
3,Mancherial,338407605,243146570,437644812,648893681,0.242351
4,Medchal_Malkajgiri,12543288516,9283678592,18200892201,24043523530,0.242212


## How does the revenue generated from document registration compare to the revenue generated from e-stamp challans across districts? List down the top 5 districts where e-stamps revenue contributes significantly more to the revenue than the documents in FY 2022?

In [161]:
doc_vs_chal = fact_stamps.groupby(['district','fiscal_year']).agg({'documents_registered_rev':'sum','estamps_challans_rev':'sum'}).reset_index()
doc_vs_chal = doc_vs_chal.pivot(index='district', columns='fiscal_year',values=['documents_registered_rev','estamps_challans_rev']).reset_index()
doc_vs_chal['diff_2022'] = doc_vs_chal[('documents_registered_rev',2022)]-doc_vs_chal[('estamps_challans_rev',2022)]
doc_vs_chal = doc_vs_chal.sort_values('diff_2022').reset_index()
top_5 = doc_vs_chal.head(5)
top_5[[('district',''),('documents_registered_rev', 2022),('estamps_challans_rev', 2022),('diff_2022','')]]

,district,documents_registered_rev,estamps_challans_rev,diff_2022
fiscal_year,,2022,2022,
0,Rangareddy,37697750946,38349357618,-651606672
1,Hyderabad,14266012441,14374315032,-108302591
2,Khammam,1971647539,2002220314,-30572775
3,Hanumakonda,2817238587,2846856844,-29618257
4,Yadadri Bhuvanagiri,2167480603,2185270667,-17790064


## Is there any alteration of e-Stamp challan count and document registration count pattern since the implementation of e-Stamp challan?


In [188]:
alteration = fact_stamps[['district','fiscal_year','documents_registered_cnt','estamps_challans_cnt']]
alteration = alteration[alteration['estamps_challans_cnt'] > 0]
alteration['pattern'] = alteration['estamps_challans_cnt'] - alteration['documents_registered_cnt'] 
alteration.head(10).reset_index(drop=True)

,district,fiscal_year,documents_registered_cnt,estamps_challans_cnt,pattern
0,Mahabubnagar,2020,1232,1403,171
1,Siddipet,2020,986,947,-39
2,Rajanna Sircilla,2020,317,313,-4
3,Mahabubabad,2020,229,290,61
4,Nalgonda,2020,1741,2087,346
5,Yadadri Bhuvanagiri,2020,1891,2487,596
6,Karimnagar,2020,955,1082,127
7,Peddapalli,2020,288,325,37
8,Hyderabad,2020,2997,3121,124
9,Nagarkurnool,2020,826,863,37


## There is alteration in document registration count pattern since implementation of e-challan.The count of document registration and estamps challan are similar. They are correlated to each other.

## Categorize districts into three segments based on their stamp registration revenue generation during the fiscal year 2021 to 2022.

In [326]:
categories = fact_stamps[fact_stamps['fiscal_year'] == 2021]
categories = categories[['district','fiscal_year','estamps_challans_rev','documents_registered_rev']]
categories = categories.groupby('district').agg({'estamps_challans_rev':'sum','documents_registered_rev':'sum'}).reset_index()
categories['total_revenue'] = categories['estamps_challans_rev'] + categories['documents_registered_rev']
categories=categories.sort_values('total_revenue',ascending = False).reset_index(drop = True).reset_index(drop=True)

# categorization the districts based on their revenue generation.(tier1,tier2,tier3)

def cat_function(row):
    if row.name >=0 and row.name <=10:
        return 'tier1'
    elif row.name >10 and row.name <=20:
        return 'tier2'
    else:
        return 'tier3'

categories['category'] = categories.apply(cat_function,axis = 1)
categories = categories.groupby(['category','district']).sum('total_revenue')
categories

estamps_challans_rev  \
category district                                      
tier1    Hanumakonda                      2112813388   
         Hyderabad                       10893677949   
         Karimnagar                       1078423581   
         Khammam                          1644624288   
         Mahabubnagar                      899612434   
         Medchal_Malkajgiri              18747892104   
         Nalgonda                         1221278894   
         Nizamabad                        1165209994   
         Rangareddy                      32332939769   
         Sangareddy                       6522436259   
         Yadadri Bhuvanagiri              1616599565   
tier2    Adilabad                          332597037   
         Jagtial                           443715297   
         Kamareddy                         380467770   
         Mancherial                        444256231   
         Medak                             482267731   
         Nagarkurnool                      378375339   
         Peddapalli                        508023769   
         Siddipet                          746741914   
         Suryapet                          792850936   
         Vikarabad                         387426499   
tier3    Bhadradri Kothagudem              254379427   
         Jangoan                           261658853   
         Jogulamba Gadwal                  272076616   
         Kumurambheem Asifabad              58294715   
         Mahabubabad                       264393331   
         Mulugu                            148856779   
         Narayanpet                        205291521   
         Nirmal                            293878065   
         Rajanna Sircilla                  294316478   
         Wanaparthy                        306488316   
         Warangal                          173457579   

                                documents_registered_rev  total_revenue  
category district                                                        
tier1    Hanumakonda                          2077808629     4190622017  
         Hyderabad                           10711890174    21605568123  
         Karimnagar                           1068655123     2147078704  
         Khammam                              1643522261     3288146549  
         Mahabubnagar                          893998095     1793610529  
         Medchal_Malkajgiri                  18200892201    36948784305  
         Nalgonda                             1216066394     2437345288  
         Nizamabad                            1151056746     2316266740  
         Rangareddy                          31865475990    64198415759  
         Sangareddy                           6344385117    12866821376  
         Yadadri Bhuvanagiri                  1619590278     3236189843  
tier2    Adilabad                              324085539      656682576  
         Jagtial                               440627967      884343264  
         Kamareddy                             379949388      760417158  
         Mancherial                            437644812      881901043  
         Medak                                 428017430      910285161  
         Nagarkurnool                          377760177      756135516  
         Peddapalli                            500011727     1008035496  
         Siddipet                              734228894     1480970808  
         Suryapet                              777314773     1570165709  
         Vikarabad                             373227540      760654039  
tier3    Bhadradri Kothagudem                  241537476      495916903  
         Jangoan                               259887134      521545987  
         Jogulamba Gadwal                      272675969      544752585  
         Kumurambheem Asifabad                  61025705      119320420  
         Mahabubabad                           257524461      521917792  
         Mulugu                                14726207

## Investigate whether there is any correlation between vehicle sales and specific months or seasons in different districts. Are there any months or seasons that consistently show higher sale rates, and if yes, what could be the driving factors?

In [328]:
fact_transport.head(3)

,dist_code,month,fuel_type_petrol,fuel_type_diesel,fuel_type_electric,fuel_type_others,vehicleClass_MotorCycle,vehicleClass_MotorCar,vehicleClass_AutoRickshaw,vehicleClass_Agriculture,vehicleClass_others,seatCapacity_1_to_3,seatCapacity_4_to_6,seatCapacity_above_6,Brand_new_vehicles,Pre-owned_vehicles,category_Non-Transport,category_Transport
0,15_1,2019-04-01,17910,3011,76,22,15308,4429,0,4,1278,16110,4182,717,19542,1477,19856,1163
1,18_2,2019-04-01,3066,306,6,0,2995,142,49,64,128,3156,189,33,3322,56,3203,175
2,20_3,2019-04-01,1577,215,0,0,1546,79,29,21,117,1683,104,5,1751,41,1648,144


In [376]:
fact_transport = pd.merge(left=fact_transport,right=districts,on='dist_code')


fact_transport[['dist_code','district']].head(5)

,dist_code,district
0,15_1,Rangareddy
1,15_1,Rangareddy
2,15_1,Rangareddy
3,15_1,Rangareddy
4,15_1,Rangareddy


In [566]:
#fact_transport = pd.merge(left=fact_transport,right=dates,on='month')
fact_transport.head(5)

,dist_code,month,fuel_type_petrol,fuel_type_diesel,fuel_type_electric,fuel_type_others,vehicleClass_MotorCycle,vehicleClass_MotorCar,vehicleClass_AutoRickshaw,vehicleClass_Agriculture,...,seatCapacity_4_to_6,seatCapacity_above_6,Brand_new_vehicles,Pre-owned_vehicles,category_Non-Transport,category_Transport,district,Mmm,quarter,fiscal_year
0,15_1,2019-04-01,17910,3011,76,22,15308,4429,0,4,...,4182,717,19542,1477,19856,1163,Rangareddy,Apr,Q1,2019
1,18_2,2019-04-01,3066,306,6,0,2995,142,49,64,...,189,33,3322,56,3203,175,Kamareddy,Apr,Q1,2019
2,20_3,2019-04-01,1577,215,0,0,1546,79,29,21,...,104,5,1751,41,1648,144,Rajanna Sircilla,Apr,Q1,2019
3,21_3,2019-04-01,1961,281,2,0,1939,72,72,48,...,146,16,2209,35,2075,169,Jangoan,Apr,Q1,2019
4,21_7,2019-04-01,1552,309,0,0,1512,76,69,109,...,145,20,1820,41,1701,160,Jayashankar Bhupalpally,Apr,Q1,2019


## Investigate whether there is any correlation between vehicle sales and specific months or seasons in different districts. Are there any months or seasons that consistently show higher sale rates, and if yes, what could be the driving factors?

In [502]:
sales_by_month = fact_transport.groupby(['district','Mmm']).agg({'Brand_new_vehicles':'sum','Pre-owned_vehicles':'sum'})
sales_by_month = sales_by_month.reset_index()
sales_by_month['total_sales'] = sales_by_month['Brand_new_vehicles'] + sales_by_month['Pre-owned_vehicles']
sales_by_month=sales_by_month.pivot(index='district',columns='Mmm',values='total_sales')
sales_by_month=sales_by_month.reset_index()
sales_by_month = sales_by_month[['district','Apr','May','Jun','Jul','Aug','Sep','Oct','Nov','Dec','Jan','Feb','Mar']]
    
sales_by_month.head()


Mmm,district,Apr,May,Jun,Jul,Aug,Sep,Oct,Nov,Dec,Jan,Feb,Mar
0,Adilabad,5536,5855,6131,6242,5918,4912,9836,7415,5467,6492,7475,8038
1,Bhadradri Kothagudem,10426,11471,14085,11829,10114,9626,17147,11067,9880,14733,12423,13987
2,Hyderabad,74320,68180,97890,95354,105226,93040,124646,102456,87164,95090,91332,108803
3,Jagtial,7344,8493,13480,11765,10781,10060,14707,10592,9454,9596,10410,9860
4,Jangoan,4809,5302,7471,6367,5971,6091,7621,6848,5543,6545,5654,6076


### Insight 1 -> October and November often coincide with various festivals and celebrations, such as Diwali and Dussehra, which are associated with increased consumer spending. During festive seasons, people may be more inclined to make big purchases like vehicles, leading to higher sales.

### Insight 2 ->  In India, the financial year ends in March, and individuals and businesses often make purchases during this time to take advantage of tax benefits or to utilize remaining budgets. This could lead to increased vehicle sales as businesses invest in vehicles for their operations or individuals purchase vehicles for personal use.

## How does the distribution of vehicles vary by vehicle class (MotorCycle, MotorCar, AutoRickshaw, Agriculture) across different districts? Are there any districts with a predominant preference for a specific vehicle class? Consider FY 2022 for analysis.

In [503]:
veh_2022 = fact_transport[fact_transport['fiscal_year']==2022]
veh_2022 = veh_2022.groupby('district').agg({'vehicleClass_MotorCycle':'sum',
                                             'vehicleClass_MotorCar':'sum',
                                             'vehicleClass_AutoRickshaw':'sum',
                                            'vehicleClass_Agriculture':'sum'})
veh_2022 = veh_2022.reset_index()
veh_2022.head()

,district,vehicleClass_MotorCycle,vehicleClass_MotorCar,vehicleClass_AutoRickshaw,vehicleClass_Agriculture
0,Adilabad,10410,1416,368,467
1,Bhadradri Kothagudem,17022,3157,1388,2013
2,Hyderabad,206819,51447,8397,20
3,Jagtial,13639,2055,272,858
4,Jangoan,8617,1177,397,1225


### insight 1 -> Preference for Two-Wheelers: 

#### Cultural factors and lifestyle preferences  play major role. In many Indian households, owning a two-wheeler is considered essential, and it may be the preferred mode of transportation for individuals and families, even if they also own four-wheelers.

### insight 2 -> Predominant preferences of particular vehicle class: 
#### 1) Hyderabad, Rangareddy, Medchal_Malkajgiri are rural areas in Telangana. Rangareddy is the highest populated district in hyderabad. From the data we can observe that vehicleClass_MotorCycle and  vehicleClass_MotorCar has highest sales. While Agriculture vehicle sales are lesser by a larger margin.

#### 2) Bhadradri Kothagudem, Jayashankar Bhupalpally, Medak, Siddipet, Suryapet, Yadadri Bhuvanagiri, These districts have a significant portion of their economy dependent on agriculture. The presence of fertile land and favorable climatic conditions for cultivation encourages extensive agricultural activities, which in turn increases the demand for agricultural machinery and vehicles.



## List down the top 3 and bottom 3 districts that have shown the highest and lowest vehicle sales growth during FY 2022 compared to FY 2021? (Consider and compare categories: Petrol, Diesel and Electric)

In [504]:
veh_sales_growth = fact_transport[(fact_transport['fiscal_year'] == 2021) | (fact_transport['fiscal_year'] == 2022)]
veh_sales_growth = veh_sales_growth.groupby(['district','fiscal_year']).agg({'fuel_type_petrol':'sum','fuel_type_diesel':'sum',
                                                         'fuel_type_electric':'sum'}).reset_index()
veh_sales_growth.head()

,district,fiscal_year,fuel_type_petrol,fuel_type_diesel,fuel_type_electric
0,Adilabad,2021,16592,2267,277
1,Adilabad,2022,11212,2523,333
2,Bhadradri Kothagudem,2021,27650,5739,76
3,Bhadradri Kothagudem,2022,19331,5712,290
4,Hyderabad,2021,228817,21195,8929


In [505]:
veh_sales_2021 = fact_transport[fact_transport['fiscal_year'] == 2021]
veh_sales_2021 = veh_sales_2021.groupby('district').agg({'fuel_type_petrol':'sum','fuel_type_diesel':'sum','fuel_type_electric':'sum'}
                                      ).reset_index()
veh_sales_2022 = fact_transport[fact_transport['fiscal_year'] == 2022]
veh_sales_2022 = veh_sales_2022.groupby('district').agg({'fuel_type_petrol':'sum','fuel_type_diesel':'sum','fuel_type_electric':'sum'}
                                      ).reset_index()


growth = pd.merge(left = veh_sales_2021, right = veh_sales_2022, on='district')
growth['total_2021'] = growth['fuel_type_petrol_x'] + growth['fuel_type_diesel_x']+growth['fuel_type_electric_x']
growth['total_2022'] = growth['fuel_type_petrol_y'] + growth['fuel_type_diesel_y']+growth['fuel_type_electric_y']


growth['difference'] = growth['total_2022'] - growth['total_2021']
growth['petrol_growth'] =  growth['fuel_type_petrol_y'] - growth['fuel_type_petrol_x']
growth['diesel_growth'] = growth['fuel_type_diesel_y'] - growth['fuel_type_diesel_x'] 
growth['electric_growth'] = growth['fuel_type_electric_y'] - growth['fuel_type_electric_x']

growth = growth[['district','total_2021','total_2022','difference','petrol_growth','diesel_growth','electric_growth']]
growth.head()

,district,total_2021,total_2022,difference,petrol_growth,diesel_growth,electric_growth
0,Adilabad,19136,14068,-5068,-5380,256,56
1,Bhadradri Kothagudem,33465,25333,-8132,-8319,-27,214
2,Hyderabad,258941,271211,12270,2349,-1176,11097
3,Jagtial,28226,17739,-10487,-10109,-611,233
4,Jangoan,17071,12160,-4911,-4323,-694,106


## INSIGHTS

### insight -> 1: There is a decline in vehicle sales in most of the districts. Rural areas has seen rise in vehicle sales between FY 2021 and FY 2022. 


### insight -> 2: RISE IN ELECTRIC VEHICLE SALE:
#### 1-> Increased competition among electric vehicle manufacturers, along with the entry of new players into the electric vehicle market, may have led to a wider selection of electric vehicle models, competitive pricing, and promotional offers, driving higher sales.

#### 2 -> Government policies promoting electric vehicles (EVs), such as subsidies, tax incentives, and rebates, could have encouraged consumers and businesses to switch to electric vehicles. These incentives may include subsidies for EV purchases, reduced taxes on EVs, or infrastructure development for charging stations.

#### 3-> Increased awareness and education about the benefits of electric vehicles, such as lower operating costs, reduced emissions, and environmental sustainability, may have influenced consumer preferences and purchasing decisions.

# Ts-Ipass (Telangana State Industrial Project Approval and Self Certification System)

In [541]:
fact_ts_ipass = pd.read_csv('/Users/udayreddy/Documents/telangana project files/C7 Input Files_/dataset/fact_TS_iPASS.csv')
fact_ts_ipass = pd.merge(left = fact_ts_ipass, right = districts, on ='dist_code')
fact_ts_ipass['month'] = pd.to_datetime(fact_ts_ipass['month']).dt.strftime('%Y-%d-%m')

fact_ts_ipass=pd.merge(left=fact_ts_ipass, right = dates, on='month')
fact_ts_ipass.head()

,dist_code,month,sector,investment in cr,number_of_employees,district,Mmm,quarter,fiscal_year
0,14_1,2019-04-01,Engineering,2.3200,15,Mahabubnagar,Apr,Q1,2019
1,14_1,2019-04-01,Paper and Printing,14.4000,305,Mahabubnagar,Apr,Q1,2019
2,19_1,2019-04-01,Engineering,0.6250,13,Adilabad,Apr,Q1,2019
3,19_1,2019-04-01,"Cement, Cement & Concrete Products, Fly Ash Br...",19.3702,99,Adilabad,Apr,Q1,2019
4,19_1,2019-04-01,Food Processing,0.0980,7,Adilabad,Apr,Q1,2019


## List down the top 5 sectors that have witnessed the most significant investments in FY 2022.

In [548]:
ipass_2022 = fact_ts_ipass[fact_ts_ipass['fiscal_year'] == 2022]
ipass_2022.groupby('sector').agg({'investment in cr':'sum'}).sort_values('investment in cr',ascending = False).reset_index().head()

,sector,investment in cr
0,Plastic and Rubber,5855.6095
1,Pharmaceuticals and Chemicals,2181.6342
2,"Real Estate,Industrial Parks and IT Buildings",2127.2963
3,Solar and Other Renewable Energy,2052.9850
4,Engineering,1877.4533


## List down the top 3 districts that have attracted the most significant sector investments during FY 2019 to 2022? What factors could have led to the substantial investments in these particular districts?

In [558]:
top_3_dist = fact_ts_ipass.groupby('district').agg({'investment in cr':'sum'}).reset_index()
top_3_dist = top_3_dist.sort_values('investment in cr',ascending = False).reset_index(drop = True)
top_3_dist.head(3)

,district,investment in cr
0,Rangareddy,42706.3320
1,Sangareddy,12366.7556
2,Medchal_Malkajgiri,10394.5610
